In [1]:
import pandas
import mmengine
# info = pandas.read_pickle('data/kitti-360/kitti360_infos_all.pkl')

In [30]:
from projects.detr3d.argo2_dataset import Kitti360Dataset
from projects.configs.extended_dataset import debug_all
from projects.configs.mono_egotest import seven
import copy
from mmdet3d.utils import register_all_modules, replace_ceph_backend

In [35]:
from mmdet3d.datasets import KittiDataset,WaymoDataset,NuScenesDataset
register_all_modules()
cfg = copy.deepcopy(seven.nusc_val)
# cfg['ann_file'] = 'kitti360_infos_2013_05_28_drive_0009_sync.pkl'
cfg.pop('type')
cfg.pop('load_interval')
dataset = NuScenesDataset(**cfg)

04/27 19:41:19 - mmengine - INFO - ------------------------------
04/27 19:41:19 - mmengine - INFO - The length of the dataset: 1924
04/27 19:41:19 - mmengine - INFO - The number of instances per category in the dataset:
+----------------------+--------+
| category             | number |
+----------------------+--------+
| car                  | 6723   |
| truck                | 1388   |
| construction_vehicle | 163    |
| bus                  | 358    |
| trailer              | 354    |
| motorcycle           | 299    |
| bicycle              | 180    |
| pedestrian           | 2791   |
+----------------------+--------+


In [40]:
zs = []
for i in range(0,len(dataset),5):
    print(i)
    info = dataset[i]
    boxes = info['data_samples'].gt_instances_3d['bboxes_3d'].gravity_center
    zs.append(boxes[...,-1])
import torch
zs = torch.cat(zs)

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655
660
665
670
675
680
685
690
695
700
705
710
715
720
725
730
735
740
745
750
755
760
765
770
775
780
785
790
795
800
805
810
815
820
825
830
835
840
845
850
855
860
865
870
875
880
885
890
895
900
905
910
915
920
925
930
935
940
945
950
955
960
965
970
975
980
985
990
995
1000
1005
1010
1015
1020
1025
1030
1035
1040
1045
1050
1055
1060
1065
1070
1075
1080
1085
1090
1095
1100
1105
1110
1115
1120
1125
1130
1135
1140
1145
1150
1155
1160
1165
1170
1175
1180
1185
1190
1195
1200
1205
1210
1215
12

In [41]:
zs.mean()

tensor(0.3130)

In [10]:
from projects.detr3d.vis_zlt import visualizer_zlt
from projects.detr3d.detr3d import DETR3D
detr3d = DETR3D()
def visualize(vis,frame):
    batch_data_samples = frame['data_samples']
    batch_inputs_dict = frame['inputs']
    batch_input_metas = [batch_data_samples.metainfo]
    batch_input_metas = detr3d.add_lidar2img(batch_input_metas)
    batch_gt_instances_3d = [
        batch_data_samples.gt_instances_3d
    ]
    vis.visualize(batch_gt_instances_3d, batch_input_metas,
                    batch_inputs_dict.get('imgs', None))
vis = visualizer_zlt('debug/kitti-360')
for i in range(0,100000,1000):
    visualize(vis,dataset[i])

reference torch.Size([2, 3])
376 1408
torch.Size([1, 1, 3])
frame: 0
reference torch.Size([3, 3])
376 1408
torch.Size([1, 2, 3])
frame: 1000
reference torch.Size([3, 3])
376 1408
torch.Size([1, 2, 3])
frame: 2000
reference torch.Size([4, 3])
376 1408
torch.Size([1, 3, 3])
frame: 3000
reference torch.Size([8, 3])
376 1408
torch.Size([1, 7, 3])
frame: 4000
reference torch.Size([2, 3])
376 1408
torch.Size([1, 1, 3])
frame: 5000
reference torch.Size([9, 3])
376 1408
torch.Size([1, 8, 3])
frame: 6000
reference torch.Size([9, 3])
376 1408
torch.Size([1, 8, 3])
frame: 7000
reference torch.Size([9, 3])
376 1408
torch.Size([1, 8, 3])
frame: 8000
reference torch.Size([3, 3])
376 1408
torch.Size([1, 2, 3])
frame: 9973
reference torch.Size([4, 3])
376 1408
torch.Size([1, 3, 3])
frame: 10000
reference torch.Size([3, 3])
376 1408
torch.Size([1, 2, 3])
frame: 11000
reference torch.Size([9, 3])
376 1408
torch.Size([1, 8, 3])
frame: 12000
reference torch.Size([2, 3])
376 1408
torch.Size([1, 1, 3])
fram

KeyboardInterrupt: 

In [13]:
from projects.detr3d.vis_zlt import visualizer_zlt
from projects.detr3d.detr3d import DETR3D
import os.path as osp
import numpy as np
from projects.detr3d.detr3d import DETR3D
detr3d = DETR3D()
vis = visualizer_zlt(debug_name='')
for i in range(8610,8620,2):
    frame = dataset[i]
    name = osp.basename(frame['data_samples'].lidar_path).split('.')[0]
    scene = frame['data_samples'].city_name
    inst = frame['data_samples'].gt_instances_3d[:2]
    batch_data_samples = frame['data_samples']
    batch_input_metas = [batch_data_samples.metainfo]
    batch_input_metas = detr3d.add_lidar2img(batch_input_metas)
    gt_bboxes_3d = inst.bboxes_3d
    corners_pt = np.array(gt_bboxes_3d.corners.view(-1, 3)).T
    print(name)
    # vis.visualize(inst, batch_input_metas,None,'test','debug/')
    vis.visualize_dataset_item(frame,name_suffix=name+'K_00', dirname='debug/kitti-360/'+scene)

0000008970
reference torch.Size([14, 3])
376 1408
torch.Size([1, 13, 3])
frame: 8610
0000008972
reference torch.Size([13, 3])
376 1408
torch.Size([1, 12, 3])
frame: 8612
0000008974
reference torch.Size([14, 3])
376 1408
torch.Size([1, 13, 3])
frame: 8614
0000008976
reference torch.Size([17, 3])
376 1408
torch.Size([1, 16, 3])
frame: 8616
0000008978
reference torch.Size([17, 3])
376 1408
torch.Size([1, 16, 3])
frame: 8618


In [5]:
from projects.detr3d.vis_zlt import visualizer_zlt
from projects.detr3d.detr3d import DETR3D
detr3d = DETR3D()
vis = visualizer_zlt(debug_name='')
def visualize(vis,frame):
    batch_data_samples = frame['data_samples']
    batch_inputs_dict = frame['inputs']
    batch_input_metas = [batch_data_samples.metainfo]
    batch_input_metas[0]['cam2img'] = [tempK]
    batch_input_metas[0]['ori_cam2img'] = [tempK]
    print(batch_input_metas[0]['ori_cam2img'])
    name = osp.basename(frame['data_samples'].lidar_path).split('.')[0]
    scene = frame['data_samples'].city_name
    batch_input_metas = detr3d.add_lidar2img(batch_input_metas)
    batch_gt_instances_3d = [
        batch_data_samples.gt_instances_3d
    ]
    vis.visualize(batch_gt_instances_3d, batch_input_metas,
                    batch_inputs_dict.get('imgs', None),name_suffix=name+'K_00', dirname='debug/kitti-360/'+scene)
for i in range(0,100,2):
    visualize(vis,dataset[i])

[array([[788.629315,   0.      , 687.158398],
       [  0.      , 786.38223 , 317.752196],
       [  0.      ,   0.      ,   1.      ]])]
reference torch.Size([2, 3])
376 1408
torch.Size([1, 1, 3])
frame: 0
[array([[788.629315,   0.      , 687.158398],
       [  0.      , 786.38223 , 317.752196],
       [  0.      ,   0.      ,   1.      ]])]
reference torch.Size([3, 3])
376 1408
torch.Size([1, 2, 3])
frame: 2
[array([[788.629315,   0.      , 687.158398],
       [  0.      , 786.38223 , 317.752196],
       [  0.      ,   0.      ,   1.      ]])]
reference torch.Size([7, 3])
376 1408
torch.Size([1, 6, 3])
frame: 4
[array([[788.629315,   0.      , 687.158398],
       [  0.      , 786.38223 , 317.752196],
       [  0.      ,   0.      ,   1.      ]])]
reference torch.Size([7, 3])
376 1408
torch.Size([1, 6, 3])
frame: 6
[array([[788.629315,   0.      , 687.158398],
       [  0.      , 786.38223 , 317.752196],
       [  0.      ,   0.      ,   1.      ]])]
reference torch.Size([7, 3])
376 1

In [19]:
from projects.detr3d.kitti360_converter import CustomCameraPerspective
cam =CustomCameraPerspective('data/kitti-360/')

cam.K

array([[552.554261,   0.      , 682.049453,   0.      ],
       [  0.      , 552.554261, 238.769549,   0.      ],
       [  0.      ,   0.      ,   1.      ,   0.      ]])